# Handy
## Transform data
When you have data (images) in the frames folder, then you have to process it.

In [21]:
%pip install opencv-python mediapipe numpy

Note: you may need to restart the kernel to use updated packages.


In [22]:
from os import path


if not path.exists("frames/"):
    print("Frames directory not found!")
    exit(-1)

In [23]:
# Create CSV file if not exists
from os import path
import csv


if not path.exists("result.csv"):
    num_columns = 33

    # Create a list of column names
    column_names = ["class_name"] + [f"{coord}{i}" for i in range(1, num_columns + 1) for coord in ["x", "y", "z", "v"]]


    with open("result.csv", mode="w", newline="") as f:
        csv_writer = csv.writer(f, delimiter=",", quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerow(column_names)

In [24]:
POSES = {
    '0': '0.png',
    '1': '1.png',
    '2': '2.png',
    '3': '3.png',
    '4': '4.png',
    '5': '5.png',
    '6': '6.png',
    '7': '7.png',
    '8': '8.png',
    '9': '9.png',
    '10': '10.png',
    '11': '11.png',
    '12': '12.png',
}


HANDY_WINDOW = "Handy"

In [25]:
import numpy as np


def calculate_angle(a, b, c):
    a = np.array(a)  # First
    b = np.array(b)  # Mid - the angle corner
    c = np.array(c)  # End

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(
        a[1] - b[1], a[0] - b[0]
    )
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

def calculate_angle_from_obj(a, b, c):
    a = (a.x, a.y)  # First
    b = (b.x, b.y)  # Mid - the angle corner
    c = (c.x, c.y)  # End

    return calculate_angle(a, b, c)

In [26]:
from os import path
import time
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

def handle_frame(frame: cv2.typing.MatLike, holistic, class_name: str) -> bool:
    # Recolor Feed
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False

    # Make Detections
    results = holistic.process(image)

    if results.pose_landmarks is None:
        return False # It means: no detections

    # Pose Detections
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_holistic.POSE_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=1, circle_radius=2),
        mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=1, circle_radius=1),
    )
    if results.pose_landmarks is not None:
        # Calculate the angles
        # See train/angles.png for more info

        landmarks = results.pose_landmarks.landmark

        angles = []
        # Angle 0
        angles.append(
            calculate_angle_from_obj(landmarks[12], landmarks[14], landmarks[16])
        )
        # Angle 1
        angles.append(
            calculate_angle_from_obj(landmarks[11], landmarks[13], landmarks[15])
        )
        # Angle 2
        angles.append(
            calculate_angle_from_obj(landmarks[14], landmarks[12], landmarks[24])
        )
        # Angle 3
        angles.append(
            calculate_angle_from_obj(landmarks[13], landmarks[11], landmarks[23])
        )

        for index, angle in enumerate(angles):
            cv2.putText(image, str(int(angle)), (0, 30 * (index + 1)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 1)
        time.sleep(0.05)

    # # class_name,x1,y1,z1,v1,x2,y2,z2,v2,... x33,y33,z33,v33
    try:

        data = np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in results.pose_landmarks.landmark]).flatten()
        row = np.concatenate(([class_name], data))


        with open("result.csv", mode="a", newline="") as f:
            csv_writer = csv.writer(f, delimiter=",", quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(row)
    except Exception as err:
        print(err)

    cv2.imshow(HANDY_WINDOW, image)
    # cv2.imshow(HANDY_MODEL_WINDOW, model_frame)
    return True

In [27]:
import os

# Prepare filelist
items = os.listdir("frames/")
items = [item for item in items if item.endswith(".png")]
items.sort()

def custom_sort_key(s):
    parts = s.split("_")
    x = int(parts[0])
    y = int(parts[1].split(".")[0])  # Remove the ".png" extension and convert to int
    return (x, y)

# Sort the array using the custom sorting key
items = sorted(items, key=custom_sort_key)


# The same amount of items per group
class_counts = {}
for item in items:
    class_name = item.split('_')[0]
    if class_name in class_counts:
        class_counts[class_name] += 1
    else:
        class_counts[class_name] = 1
min_count = min(class_counts.values())

# Balance the counts - there should be an equal amount of each item
class_count_tracker = {key: 0 for key in class_counts}
items = [input_string for input_string in items if class_count_tracker[input_string.split('_')[0]] < min_count and not (class_count_tracker.update({input_string.split('_')[0]: class_count_tracker[input_string.split('_')[0]] + 1}) or False)]

assert(len(items) / min_count == len(class_counts))

items

['0_0.png',
 '0_1.png',
 '0_2.png',
 '0_3.png',
 '0_4.png',
 '0_5.png',
 '0_6.png',
 '0_7.png',
 '0_8.png',
 '0_9.png',
 '0_10.png',
 '0_11.png',
 '0_12.png',
 '0_13.png',
 '0_14.png',
 '0_15.png',
 '0_16.png',
 '0_17.png',
 '0_18.png',
 '0_19.png',
 '0_20.png',
 '0_21.png',
 '0_22.png',
 '0_23.png',
 '0_24.png',
 '0_25.png',
 '0_26.png',
 '0_27.png',
 '0_28.png',
 '0_29.png',
 '0_30.png',
 '0_31.png',
 '0_32.png',
 '0_33.png',
 '0_34.png',
 '0_35.png',
 '0_36.png',
 '0_37.png',
 '0_38.png',
 '0_39.png',
 '0_40.png',
 '0_41.png',
 '0_42.png',
 '0_43.png',
 '0_44.png',
 '0_45.png',
 '0_46.png',
 '0_47.png',
 '0_48.png',
 '0_49.png',
 '0_50.png',
 '0_51.png',
 '0_52.png',
 '0_53.png',
 '0_54.png',
 '0_55.png',
 '0_56.png',
 '0_57.png',
 '0_58.png',
 '0_59.png',
 '0_60.png',
 '0_61.png',
 '0_62.png',
 '0_63.png',
 '0_64.png',
 '0_65.png',
 '0_66.png',
 '0_67.png',
 '0_68.png',
 '0_69.png',
 '0_70.png',
 '0_71.png',
 '0_72.png',
 '0_73.png',
 '0_74.png',
 '0_75.png',
 '0_76.png',
 '0_77.pn

In [28]:
from datetime import datetime, timedelta
import cv2
import mediapipe as mp
import numpy as np

mp_holistic = mp.solutions.holistic

cv2.namedWindow(HANDY_WINDOW)

with mp_holistic.Holistic(
    min_detection_confidence=0.7, min_tracking_confidence=0.95
) as holistic:
    last_process = datetime.now()

    for image in items:
        frame = cv2.imread(path.join("frames", image))
        success = handle_frame(frame, holistic, image.split("_")[0]) # Get image class_name from the filename (e.g. 0_1.png where 0 is the class_name and 1 is the index)
        
        if success:
            print(f"Frame {image} processed in {(datetime.now() - last_process).total_seconds()}s")
        
        else:
            print(f"Frame wasn't processed - no detection")
        if (cv2.waitKey(1) & 0xFF == ord("q")):
            break
        last_process = datetime.now()


cv2.destroyAllWindows()

Frame 0_0.png processed in 0.299879s
Frame 0_1.png processed in 0.111448s
Frame 0_2.png processed in 0.127803s
Frame 0_3.png processed in 0.113167s
Frame 0_4.png processed in 0.11058s
Frame 0_5.png processed in 0.116772s
Frame 0_6.png processed in 0.10943s
Frame 0_7.png processed in 0.120024s
Frame 0_8.png processed in 0.108094s
Frame 0_9.png processed in 0.124116s


Frame 0_10.png processed in 0.114045s
Frame 0_11.png processed in 0.128431s
Frame 0_12.png processed in 0.111484s
Frame 0_13.png processed in 0.123176s
Frame 0_14.png processed in 0.124887s
Frame 0_15.png processed in 0.112387s
Frame 0_16.png processed in 0.108739s
Frame 0_17.png processed in 0.114524s
Frame 0_18.png processed in 0.115209s
Frame 0_19.png processed in 0.117517s
Frame 0_20.png processed in 0.118509s
Frame 0_21.png processed in 0.105961s
Frame 0_22.png processed in 0.105794s
Frame 0_23.png processed in 0.120223s
Frame 0_24.png processed in 0.108093s
Frame 0_25.png processed in 0.113053s
Frame 0_26.png processed in 0.123196s
Frame 0_27.png processed in 0.11056s
Frame 0_28.png processed in 0.124294s
Frame 0_29.png processed in 0.109781s
Frame 0_30.png processed in 0.111709s
Frame 0_31.png processed in 0.11597s
Frame 0_32.png processed in 0.127472s
Frame 0_33.png processed in 0.107239s
Frame 0_34.png processed in 0.113973s
Frame 0_35.png processed in 0.113261s
Frame 0_36.png

In [ ]:
# Get number of data per each class

all_frames = os.listdir("frames/")
result = {}

for class_name in POSES:
    # Use a list comprehension to filter files that start with "0_"
    matching_files = [filename for filename in all_frames if filename.startswith(f"{class_name}_")]

    # Get the number of matching files
    number_of_matching_files = len(matching_files)
    result[class_name] = number_of_matching_files

print(result)


{'0': 2000, '1': 200, '2': 200, '3': 200, '4': 200, '5': 200, '6': 200, '7': 200, '8': 200, '9': 200, '10': 0, '11': 0, '12': 0}
